In [ ]:
import itertools
import numpy
from scipy import optimize

In [ ]:
sequence_counts = [
    ("ATC", 18000),
    ("CAG", 9000),
    ("TAG", 9000),
    ("CCA", 8000),
    ("AAC", 7000),
    ("AAT", 100),
    ("CAT", 200)
]

In [ ]:
ERROR_PROBABILITY = 0.25
THRESHOLD = 0
MAX_DISTANCE = 3

ERROR_PROBABILITIES = [1 - ERROR_PROBABILITY]
for distance in range(1, MAX_DISTANCE + 1):
    ERROR_PROBABILITIES.append(numpy.power(ERROR_PROBABILITY, distance))

In [ ]:
def prune_sequence_counts(sequence_counts):
    
    sequences_to_prune = []

    for sequence_index, (sequence, count) in enumerate(sequence_counts):
        if count < THRESHOLD:
            sequences_to_prune.append(sequence_index)
    
    for sequence_index in reversed(sequences_to_prune):
        del sequence_counts[sequence_index]
    
    return sequence_counts

def hamming(s1, s2):
    """Calculate the Hamming distance between two bit strings"""
    assert len(s1) == len(s2)
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

In [ ]:
sequence_counts = prune_sequence_counts(sequence_counts)

In [ ]:
sequence_counts = [
    ("ATC", 18000),
    ("CAG", 9000),
    ("TAG", 9000),
    ("CCA", 8000),
    ("AAC", 7000),
    ("AAT", 100),
    ("CAT", 200)
]

sequence_index = 0

while sequence_index < len(sequence_counts) - 1:
    
    sequence_subset = {0: [sequence_counts[sequence_index]]}
    
    sequence_search_space = sequence_counts[sequence_index+1:]
    
    for distance in range(1, MAX_DISTANCE + 1):
        
        sequences_at_distance = []
        
        for current_sequence_counts in sequence_subset[distance - 1]:
            
            sequences_at_distance_indices = []
            
            for s_index, s in enumerate(sequence_search_space):
                if hamming(s[0], current_sequence_counts[0]) == 1:
                    sequences_at_distance.append(s)
                    sequences_at_distance_indices.append(s_index)
        
            for sequence_to_delete_index in reversed(sequences_at_distance_indices):
                del sequence_search_space[sequence_to_delete_index]
        
        sequence_subset[distance] = sequences_at_distance
    
    sequence_subset = list(itertools.chain.from_iterable([value for key, value in sequence_subset.items()]))
    
    num_sequences = len(sequence_subset)
    
    if num_sequences > 1:
    
        probabilities = numpy.eye(num_sequences) * (1 - ERROR_PROBABILITY)
        counts = numpy.zeros((num_sequences,))
        
        for sequence_row_index in range(num_sequences):
            
            counts[sequence_row_index] = sequence_subset[sequence_row_index][1]
            
            row_sequence = sequence_subset[sequence_row_index][0]
            
            for sequence_column_index in range(sequence_row_index + 1, num_sequences):
            
                column_sequence = sequence_subset[sequence_column_index][0]
                
                distance = hamming(row_sequence, column_sequence)
                
                probabilities[sequence_row_index, sequence_column_index] = ERROR_PROBABILITIES[distance]
                probabilities[sequence_column_index, sequence_row_index] = ERROR_PROBABILITIES[distance]
    
        results = optimize.lsq_linear(probabilities, counts, bounds=(0, numpy.inf))

        updated_counts = results.x

        for sequence_to_update_index, (sequence_to_update, count) in enumerate(sequence_subset):
            for sequence_index, (sequence, _) in enumerate(sequence_counts):
                if sequence_to_update == sequence:
                    sequence_counts[sequence_index] = (sequence_to_update, updated_counts[sequence_to_update_index])
                    break
    
    sequence_counts = prune_sequence_counts(sequence_counts)
    sequence_index += 1

In [ ]:
sequence_counts